IMPORTARE EL CSV DE LOS DATOS HISTORICOS DEL CLIMA, DESCARGADO DESDE LA API DE LA NASA.

CON ESTO PODRE HACER UN MERGE USANDO COMO CLAVE LA FECHA PARA TENER LA INFORMACION DETALLADA DEL ESTADO DEL TIEMPO DEL DIA DEL SISENTRO

EDIT: TODAVIA NO PUDE BAJAR LOS DATOS USANDO LA API

In [72]:
import pandas as pd

In [73]:
siniestros = 'victimas-siniestros.csv'
sin_df = pd.read_csv(siniestros)

C:\Users\Sistemas\AppData\Local\Temp\ipykernel_11204\1239150759.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  sin_df = pd.read_csv(siniestros)


In [74]:
clima = 'clima.csv'
clima_df = pd.read_csv(clima)

In [75]:
clima_df.head()

,YEAR,MO,DY,QV2M,RH2M,PRECTOTCORR,T2M,TS,WS2M,CLRSKY_SFC_SW_DWN,PS,WS10M
0,2015,1,1,8.54,64.94,1.78,19.02,19.62,4.21,9.10,100.53,6.15
1,2015,1,2,7.51,62.69,0.00,18.11,19.20,2.98,9.55,101.28,4.42
2,2015,1,3,8.97,60.44,0.00,21.31,21.56,2.61,9.37,101.22,3.95
3,2015,1,4,9.46,64.94,0.00,20.99,22.71,2.60,9.30,101.39,3.70
4,2015,1,5,12.45,64.19,0.00,25.64,26.23,3.27,9.10,100.89,4.81


SOLO RESTA NORMALIZAR LA FECHA CON EL DIA, MES Y AÑO EN LA MISMA COLUMNA Y HACER EL MERGE CON ESE VALOR

In [76]:
sin_df.head()

,causa,rol,tipo,sexo,edad,mes,periodo,fecha,hora,lugar_hecho,...,semestre,x,y,geom,cantidad_victimas,comuna,geom_3857,tipo_colision1,participantes_victimas,participantes_acusados
0,homicidio,conductor,moto,NaN,NaN,2.0,2015,2/14/2015,19:00:00,cafayate y severo garcia grande de zequeira,...,1,-58.508658,-34.653489,0101000020E610000057E042B51B414DC008419885A553...,1,9.0,0101000020110F0000DE9B958180D858C1EC802966CD68...,motovehiculo - vehiculo,moto,automovil
1,homicidio,NaN,NaN,NaN,NaN,2.0,2015,2/25/2015,3:00:00,"lugones, leopoldo av. y udaondo, guillermo av.",...,1,-58.447680,-34.543513,0101000020E6100000CC4C4C904D394DC0FAF382D99145...,1,13.0,0101000020110F0000BFBC457BDFD158C1E1DDA818C14B...,NaN,NaN,NaN
2,homicidio,peaton,peaton,femenino,NaN,2.0,2015,2/27/2015,8:00:00,avda jujuy y avda independencia,...,1,-58.403516,-34.618839,0101000020E6100000FBC35B67A6334DC03F854E1F364F...,1,3.0,0101000020110F0000F8067D6812CD58C189755F66A55F...,peaton - vehiculo,peaton,"""transporte publico"""
3,homicidio,NaN,NaN,NaN,NaN,3.0,2015,03-02-15,18:30:00,lavalle 1730,...,1,-58.391329,-34.603168,0101000020E6100000312FB20D17324DC064587D9E344D...,1,1.0,0101000020110F00005621683DBFCB58C10B59E69B815B...,NaN,NaN,NaN
4,homicidio,pasajero,camion,masculino,NaN,4.0,2015,04-09-15,1:20:00,ave salvador m del carril 2434,...,1,-58.492895,-34.587489,0101000020E61000009BDD7231173F4DC02D782AD7324B...,1,15.0,0101000020110F00006F2F76D4C9D658C1750825735D57...,vehiculo - vehiculo,camion,camion


FUNCIONES PARA MODIFICAR LA FECHA Y PODER HACER MERGE CON EL SET DE DATOS DEL CLIMA

In [132]:
def crear_fecha(fecha):

    if "/" in fecha:
        return fecha.split("/")
    elif "-" in fecha:
        return fecha.split("-")
        #aux = fecha[0]
        #fecha[0] = fecha[1]
        #fecha[1] = aux
        #return fecha

def limpiar_fecha(fecha):
    
    d = int(fecha[1])
    m = int(fecha[0])
    a = int(fecha[2])
    
    if a<2000:
        a = 2000+a
    
    if m>12:
        aux = m
        m = a
        a = aux
    
    return [d,m,a]

def dia(d):
    return d[0]

def mes(d):
    return d[1]

def anio(d):
    return d[2]


#sin_df['dia'] = sin_df.dia.str.split("-")


In [122]:
sin_df.fecha.isnull().sum()

0

In [123]:
sin_df = sin_df.dropna(subset=['fecha'])
sin_df.fecha.isnull().sum()

0

In [124]:
sin_df['fecha_norm'] = sin_df['fecha'].apply(crear_fecha)
sin_df['fecha_norm'] = sin_df['fecha_norm'].apply(limpiar_fecha)

In [133]:
sin_df['d'] = sin_df['fecha_norm'].apply(dia)
sin_df['m'] = sin_df['fecha_norm'].apply(mes)
sin_df['a'] = sin_df['fecha_norm'].apply(anio)

YA TENGO EL DATAFRAME LISTO PARA HACER EL MERGE

In [151]:
sin_df = sin_df.merge(clima_df,how='left',left_on=['d','m','a'],right_on=['DY','MO','YEAR'])
sin_df.head(2)

,causa,rol,tipo,sexo,edad,mes,periodo,fecha,hora,lugar_hecho,...,DY_y,QV2M_y,RH2M_y,PRECTOTCORR_y,T2M_y,TS_y,WS2M_y,CLRSKY_SFC_SW_DWN_y,PS_y,WS10M_y
0,homicidio,conductor,moto,NaN,NaN,2.0,2015,2/14/2015,19:00:00,cafayate y severo garcia grande de zequeira,...,14,9.03,60.06,0.00,21.71,23.18,3.62,8.08,101.66,5.15
1,homicidio,NaN,NaN,NaN,NaN,2.0,2015,2/25/2015,3:00:00,"lugones, leopoldo av. y udaondo, guillermo av.",...,25,13.79,79.12,1.26,23.12,23.25,4.02,7.29,101.14,5.94


CORROBORO CON EL PRIMER VALOR QUE LOS DATOS DEL CLIMA ESTEN BIEN

In [152]:
clima_df.loc[(clima_df['DY']==14) & (clima_df['MO']==2) & (clima_df['YEAR']==2015)]

,YEAR,MO,DY,QV2M,RH2M,PRECTOTCORR,T2M,TS,WS2M,CLRSKY_SFC_SW_DWN,PS,WS10M
44,2015,2,14,9.03,60.06,0.0,21.71,23.18,3.62,8.08,101.66,5.15


NO LO TENGO DEL TODO HECHO, PERO YA PUDE BAJAR LOS DATOS DEL CLIMA DE LA PAGINA DE LA NASA USANDO LA API, ME FALTA ENTENDERLA MEJOR PARA LLEGAR AL DATASET QUE DESCARGUE USANDO EL NAVEGADOR

A CONTINUACION ESTA EL CODIGO

In [159]:
import os, json, requests

In [160]:
locations = [(32.929, -95.770), (5, 10)]

output = r""
base_url = r"https://power.larc.nasa.gov/api/temporal/daily/point?parameters=WS2M,T2M,TS,RH2M,PS,WS10M&community=RE&longitude=-58.3999&latitude=-34.6025&start=20150101&end=20181212&format=JSON"

for latitude, longitude in locations:
    api_request_url = base_url.format(longitude=longitude, latitude=latitude)

    response = requests.get(url=api_request_url, verify=True, timeout=30.00)

    content = json.loads(response.content.decode('utf-8'))
    filename = response.headers['content-disposition'].split('filename=')[1]

    filepath = os.path.join(output, filename)
    with open(filepath, 'w') as file_object:
        json.dump(content, file_object)

In [161]:
content

{'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [-58.3999, -34.6025, 6.58]},
 'properties': {'parameter': {'WS2M': {'20150101': 7.07,
    '20150102': 5.64,
    '20150103': 4.22,
    '20150104': 4.89,
    '20150105': 4.66,
    '20150106': 4.19,
    '20150107': 4.56,
    '20150108': 2.33,
    '20150109': 5.43,
    '20150110': 3.46,
    '20150111': 3.67,
    '20150112': 3.91,
    '20150113': 5.52,
    '20150114': 3.26,
    '20150115': 5.56,
    '20150116': 4.0,
    '20150117': 5.73,
    '20150118': 4.72,
    '20150119': 6.77,
    '20150120': 6.88,
    '20150121': 4.56,
    '20150122': 5.88,
    '20150123': 5.06,
    '20150124': 3.0,
    '20150125': 1.76,
    '20150126': 2.22,
    '20150127': 3.42,
    '20150128': 7.57,
    '20150129': 6.34,
    '20150130': 4.2,
    '20150131': 4.53,
    '20150201': 4.25,
    '20150202': 4.62,
    '20150203': 5.81,
    '20150204': 5.09,
    '20150205': 3.78,
    '20150206': 4.36,
    '20150207': 4.1,
    '20150208': 3.76,
    '20150209': 